## Assignment 1 (Visual)


In [5]:
#importing libraries
import os
import sys
sys.path.append("..")

import cv2
import numpy as np
import pandas as pd

### Setup
Defining path to chosen target flower and defining functions. <br>

Includes a function (i.e, 'calc_hist') for getting the colour histogram using OpenCV and normalising them.

The second function (i.e., 'compare_hists') utilises the above to extract the histograms for all other inputted images and then compares them to the target image.

In [61]:
filepath_target_flower = os.path.join("..",
                        "..",
                        "..",
                        "..",
                        "cds-vis-data",
                        "flowers",
                        "image_0555.jpg")


target_image = cv2.imread(filepath_target_flower)

In [62]:
#function for calculating histograms for inputted image
def calc_hist(image):
    #creating empty list
    hist = []
    
    #splitting
    channels = cv2.split(image)
    
    #calculating hist for each channel
    for channel in channels:
        hist_channel = cv2.calcHist([channel], [0], None, [255], [0,256])
        #append list
        hist.append(hist_channel)
    
    #concatenating hists into 1 array --> normalise
    hist = np.concatenate(hist)
    hist_normalised = cv2.normalize(hist, hist, 0, 1.0, cv2.NORM_MINMAX)

    return hist_normalised    

In [63]:
#function for comparing histograms of target img with other image
def compare_hists(target_hist, image_path):
    #loading img
    image = cv2.imread(image_path)

    #calculating hist with previous function
    image_hist = calc_hist(image)
    
    #comparing hists
    distance = round(cv2.compareHist(target_hist, image_hist, cv2.HISTCMP_CHISQR),2)

    return distance

### Running the script
Firstly getting the colour histogram for the target image.  

Defining path to rest of images and defining filenames. Then using a for loop with the 'compare_hists' function to get the needed information. Finally, outputs the five most similar images to the target.

In [64]:
#calculating histogram for target image
target_hist = calc_hist(target_image)

In [65]:
#path to images
data_path = os.path.join("..",
                        "..",
                        "..",
                        "..",
                        "cds-vis-data",
                        "flowers")
#defining filenames
filenames = [file for file in os.listdir(data_path) if file.endswith('jpg')]


In [66]:
#list to store the (filename, distances)
results = [] 

for filename in filenames:
    img_path = os.path.join(data_path, filename)
    distance = compare_hists(target_hist, img_path)
    results.append((filename, distance))

#checking for the 5 with shortest distance
sorted_dist = sorted(results, key = lambda x: x[1])[:6] #after sorting, using slicing to select first 5 items

#creating pandas df
results_df = pd.DataFrame(sorted_dist, columns=["Filename", "Distance"])

results_df.to_csv("../out/most_similar_images.csv", index = False)

In [ ]:
#possible other way in case of target not occuring in file

#new_row = pd.DataFrame({'Filename': ['Target'], 'Distance': [0.0]})
#appending the new row to the DataFrame
#results_df = pd.concat([new_row, results_df]).reset_index(drop=True)